In [31]:
with open("access_token", "r") as file:
    access_token = file.read().strip()

import requests
import json

**1. Schema**

GitHub's API schema consists of a tree-like search structure, with each level representing sub-structures of the current node.

For example, we can request a list of repos, or we can request their individual sub-structures, like pull requests. 

In [32]:
def get_graphql_rate_limit(query, access_token):
  url = "https://api.github.com/graphql"
  headers = {"Authorization": f"Bearer {access_token}"}
  data = {"query": query}
  response = requests.post(url, json=data, headers=headers)
  rate_limit_data = response.json()["data"]["rateLimit"]
  limit = rate_limit_data["limit"]
  remaining = rate_limit_data["remaining"]
  reset_at = rate_limit_data["resetAt"]
  return limit, remaining, reset_at

num_each = 100

repos_query = """
{
  viewer {
    repositories(first: """ + str(num_each) + """) {
      nodes {
        name
      }
    }
  }
  rateLimit {
    limit
    remaining
    resetAt
  }
}
"""

pulls_query = """
{
  viewer {
    repositories(first: 1) {
      nodes {
        pullRequests(first: """ + str(num_each) + """) {
          nodes {
            title
          }
        }
      }
    }
  }
  rateLimit {
    limit
    remaining
    resetAt
  }
}
"""

comments_query = """
{
  viewer {
    repositories(first: 1) {
      nodes {
        issues(first: 1) {
          nodes {
            comments(first: """ + str(num_each) + """) {
              nodes {
                body
              }
            }
          }
        }
      }
    }
  }
  rateLimit {
    limit
    remaining
    resetAt
  }
}
"""

commits_query = """
{
  viewer {
    repositories(first: 1) {
      nodes {
        defaultBranchRef {
          target {
            ... on Commit {
              history(first: """ + str(num_each) + """) {
                nodes {
                  message
                }
              }
            }
          }
        }
      }
    }
  }
  rateLimit {
    limit
    remaining
    resetAt
  }
}
"""

repos_limit, repos_remaining, repos_reset_at = get_graphql_rate_limit(repos_query, access_token)
print("Repositories rate limit details:")
print(f"Limit: {repos_limit}")
print(f"Remaining: {repos_remaining}")
print(f"Reset At (UTC): {repos_reset_at}")
print()

pulls_limit, pulls_remaining, pulls_reset_at = get_graphql_rate_limit(pulls_query, access_token)
print("Pull requests rate limit details:")
print(f"Limit: {pulls_limit}")
print(f"Remaining: {pulls_remaining}")
print(f"Reset At (UTC): {pulls_reset_at}")
print()

comments_limit, comments_remaining, comments_reset_at = get_graphql_rate_limit(comments_query, access_token)
print("Comments rate limit details:")
print(f"Limit: {comments_limit}")
print(f"Remaining: {comments_remaining}")
print(f"Reset At (UTC): {comments_reset_at}")
print()

commits_limit, commits_remaining, commits_reset_at = get_graphql_rate_limit(commits_query, access_token)
print("Commits rate limit details:")
print(f"Limit: {commits_limit}")
print(f"Remaining: {commits_remaining}")
print(f"Reset At (UTC): {commits_reset_at}")


Repositories rate limit details:
Limit: 5000
Remaining: 4998
Reset At (UTC): 2024-06-06T20:46:24Z

Pull requests rate limit details:
Limit: 5000
Remaining: 4997
Reset At (UTC): 2024-06-06T20:46:24Z

Comments rate limit details:
Limit: 5000
Remaining: 4996
Reset At (UTC): 2024-06-06T20:46:24Z

Commits rate limit details:
Limit: 5000
Remaining: 4995
Reset At (UTC): 2024-06-06T20:46:24Z


**2. Cost**

The cost appears to be low as long as you have reasonable requests. E.g., 100 repos, 100 pulls, 100 comments, and 100 commits cost 1 each, which is very low (maybe we do actually want more, I'm just not sure yet!).

**3. Quota limits per person per hour**

5000 points per person per hour

In [35]:
query = """{
  first: repository(owner: "sdispater", name: "pendulum") {
    issue(number: 153) {
      title
      body
      createdAt
      updatedAt
      comments(first: 20) {
        nodes {
          body
          createdAt
          updatedAt
        }
      }
    }
  }
  second: repository(owner: "arrow-py", name: "arrow") {
    issue(number: 1170) {
      title
      body
      createdAt
      updatedAt
      comments(first: 20) {
        nodes {
          body
          createdAt
          updatedAt
        }
      }
    }
  }
  third: repository(owner: "python", name: "cpython") {
    issue(number: 70647) {
      title
      body
      createdAt
      updatedAt
      comments(first: 20) {
        nodes {
          body
          createdAt
          updatedAt
        }
      }
    }
  }
}
"""
url = "https://api.github.com/graphql"
headers = {"Authorization": f"Bearer {access_token}"}
data = {"query": query}
response = requests.post(url, json=data, headers=headers).json()
print(json.dumps(response, indent=4))

{
    "data": {
        "first": {
            "issue": {
                "title": "difference in days is incorrect across daylight savings",
                "body": "The date difference between 3/13/2017 and 3/12/2017 is 0 days (it should be 1 day)\r\n\r\nReproducible code:\r\n```python\r\nstart = pendulum.create(2017, 3, 12, 0, 0, 0, tz='America/Los_Angeles')\r\nfor d in range(3):\r\n    end = start.add(days=d)\r\n    day_diff = end.diff(start).in_days()\r\n    print('{} minus {} is {} days'.format(end, start, day_diff))\r\n```\r\n\r\noutput:\r\n```\r\n2017-03-12T00:00:00-08:00 minus 2017-03-12T00:00:00-08:00 is 0 days\r\n2017-03-13T00:00:00-07:00 minus 2017-03-12T00:00:00-08:00 is 0 days\r\n2017-03-14T00:00:00-07:00 minus 2017-03-12T00:00:00-08:00 is 1 days\r\n```\r\n\r\npython 3.5.2\r\npendulum==1.3.0\r\nMac OS 10.12.6",
                "createdAt": "2017-10-08T20:18:08Z",
                "updatedAt": "2017-10-21T13:25:22Z",
                "comments": {
                    "nodes"

**4. The query is above**